In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "online-retail-rfm-cohort-0d645667cef8.json"


In [3]:
credentials = service_account.Credentials.from_service_account_file(
    'online-retail-rfm-cohort-0d645667cef8.json'
)


In [4]:
client = bigquery.Client()


In [5]:
def get_query(func):
    """
    A decorator function that reads a SQL query from a file and passes it to the decorated function.

    Args:
    - func (function): A function that processes a SQL query.

    Returns:
    - read_query_from_file (function): A function that reads a SQL query from a file
      specified by `file_name`, and passes it to `func` for further processing.
    """
    def read_query_from_file(file_name):
        script_dir = os.getcwd()  
        abs_file_path = os.path.join(script_dir, 'queries', file_name)
        
        with open(abs_file_path, 'r') as file:
            query = file.read()
        return func(query)
    return read_query_from_file
    

In [6]:
@get_query
def select(query):
    """
    Executes a SQL query using Google BigQuery client and returns the result as a Pandas DataFrame.

    Args:
    - query (str): SQL query string to be executed.

    Returns:
    - pd.DataFrame: Result of the SQL query in the form of a Pandas DataFrame.
    """
    return client.query(query).to_dataframe()
    

In [7]:
select('1_select_all_f_customers.sql')


,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
0,2011-10-20 14:41:00+00:00,572092,21617,4 LILY BOTANICAL DINNER CANDLES,12,1.25,17125.0,15,male,20280.508576,United Kingdom
1,2011-10-20 14:41:00+00:00,572092,23371,SET 36 COLOUR PENCILS SPACEBOY,1,1.25,17125.0,15,male,20280.508576,United Kingdom
2,2011-10-20 14:41:00+00:00,572092,23372,SET 36 COLOUR PENCILS DOLLY GIRL,1,1.25,17125.0,15,male,20280.508576,United Kingdom
3,2011-10-20 14:41:00+00:00,572092,22979,PANTRY WASHING UP BRUSH,1,1.45,17125.0,15,male,20280.508576,United Kingdom
4,2011-10-20 14:41:00+00:00,572092,85035B,CHOCOLATE 3 WICK MORRIS BOX CANDLE,1,1.25,17125.0,15,male,20280.508576,United Kingdom
5,2011-10-20 14:41:00+00:00,572092,22632,HAND WARMER RED RETROSPOT,3,2.10,17125.0,15,male,20280.508576,United Kingdom
6,2011-10-20 14:41:00+00:00,572092,22697,GREEN REGENCY TEACUP AND SAUCER,1,2.95,17125.0,15,male,20280.508576,United Kingdom
7,2011-10-20 14:41:00+00:00,572092,22491,PACK OF 12 COLOURED PENCILS,10,0.85,17125.0,15,male,20280.508576,United Kingdom
8,2011-10-20 14:41:00+00:00,572092,22699,ROSES REGENCY TEACUP AND SAUCER,1,2.95,17125.0,15,male,20280.508576,United Kingdom
9,2011-10-20 14:41:00+00:00,572092,22699,ROSES REGENCY TEACUP AND SAUCER,1,2.95,17125.0,15,male,20280.508576,United Kingdom


#### Checking the presence of missing values ​​in all columns

In [8]:
select('2_missing_values_check.sql')


,f0_,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
0,541909,541909,541909,541909,540455,541909,541909,406829,541909,541909,541909,541909


<p style='color:green; font-weight: 800;'>The dataset has 541909 lines. The Description and CustomerID columns have 540455 and 406829 filled values, respectively. Therefore, these columns contain missing values.  
Since we will not need the Description field in further analysis, let's focus on CustomerId.</p>

#### Checking for records without missing data with invoice numbers where no CustomerId is specified to decide whether to fill in the missing data


In [9]:
select('3_check_non_missing_records_with_invoice_no_customerid.sql')


,CustomerID
0,NaN


<p style='color:green; font-weight: 800;'>According to the results of the check, we do not have data that can be relied on when filling in the missing data, we decide on the need to delete the lines in which CustomerID is NULL.</p>

#### The next step is to check the dataset for duplicates.

In [10]:
select('4_duplicates_check.sql')


,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Income,duplicates
0,2011-10-20 14:41:00+00:00,572092,22697,GREEN REGENCY TEACUP AND SAUCER,1,2.95,17125.0,20280.508576,2
1,2011-10-20 14:41:00+00:00,572092,22699,ROSES REGENCY TEACUP AND SAUCER,1,2.95,17125.0,20280.508576,2
2,2011-10-20 14:41:00+00:00,572092,21379,CAMPHOR WOOD PORTOBELLO MUSHROOM,6,1.25,17125.0,20280.508576,2
3,2011-10-20 14:41:00+00:00,572092,23071,MARIE ANTOINETTE TRINKET BOX GOLD,1,12.50,17125.0,20280.508576,2
4,2011-10-20 14:41:00+00:00,572092,22698,PINK REGENCY TEACUP AND SAUCER,1,2.95,17125.0,20280.508576,2
...,...,...,...,...,...,...,...,...,...
4874,2011-12-04 12:32:00+00:00,580469,22079,RIBBON REEL HEARTS DESIGN,2,1.65,14583.0,26632.766899,2
4875,2011-12-04 12:32:00+00:00,580469,23048,SET OF 10 LANTERNS FAIRY LIGHT STAR,1,4.15,14583.0,26632.766899,2
4876,2011-11-17 19:48:00+00:00,577128,85136A,YELLOW SHARK HELICOPTER,1,7.95,14508.0,25271.287502,2
4877,2011-11-17 19:48:00+00:00,577128,22536,MAGIC DRAWING SLATE PURDEY,1,0.42,14508.0,25271.287502,2


<p style='color:green; font-weight: 800;'>The dataset has 4,879 lines that have duplicates (i.e. a complete match across all data).  
We will remove all duplicates during the data cleaning phase</p>

#### Checking the extreme values ​​of data distributions by columns:  
- InvoiceDate
- Quantity
- Income

In [11]:
select('5_extreme_values_check.sql')


,min_date,max_date,min_qty,max_qty,min_price,max_price,min_income,max_income
0,2010-12-01 08:26:00+00:00,2011-12-09 12:50:00+00:00,-80995,80995,-11062.06,38970.0,18662.725643,45925.206366


<p style='color:green; font-weight: 800;'>TThe dataset contains data for 1 year. It includes cases when the price and quantity of goods have negative values. At the same time, only positive values ​​are displayed in the Income column.</p>

#### Let's check Quantity column with negative values.  
Could it be data about the return of the product? To do this, we will try to find lines with a positive value of the quantity of the product similar to lines with a negative value of the product.

In [12]:
select('6_quantity_negative_values_check.sql')


,InvoiceNo,date,InvoiceNo_1,date_1,CustomerID,StockCode,StockCode_1,UnitPrice,purchase_qty,returned_qty
0,539475,2010-12-19,C543290,2011-02-07,16686.0,21579,21579,2.25,2,-2
1,554992,2011-05-29,C543290,2011-02-07,16686.0,22617,22617,4.95,2,-2
2,547020,2011-03-18,C554340,2011-05-23,17017.0,20979,20979,1.25,16,-16
3,553836,2011-05-19,C554340,2011-05-23,17017.0,20979,20979,1.25,16,-16
4,558571,2011-06-30,C554340,2011-05-23,17017.0,20979,20979,1.25,16,-16
...,...,...,...,...,...,...,...,...,...,...
5595,562274,2011-08-04,C574447,2011-11-04,12704.0,POST,POST,40.00,2,-2
5596,564558,2011-08-26,C574447,2011-11-04,12704.0,POST,POST,40.00,2,-2
5597,567938,2011-09-22,C574447,2011-11-04,12704.0,POST,POST,40.00,2,-2
5598,577067,2011-11-17,C574447,2011-11-04,12704.0,POST,POST,40.00,2,-2


<p style='color:green; font-weight: 800;'>Let's check in more detail on the example of the first two lines from the obtained result. Note that we have different invoices and different products, but the purchase was made by the same buyer. At the same time, the Invoice number with the negative value of the product quantity is the same for both cases.</p>

In [13]:
select('7_if_it_product_return_checking_on_sample.sql')


,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
0,2011-02-07 10:44:00+00:00,C543290,22617,BAKING SET SPACEBOY DESIGN,-2,4.95,16686.0,15,female,21032.747844,United Kingdom
1,2011-02-07 10:44:00+00:00,C543290,21579,LOLITA DESIGN COTTON TOTE BAG,-2,2.25,16686.0,15,female,21032.747844,United Kingdom
2,2010-12-19 14:41:00+00:00,539475,21579,LOLITA DESIGN COTTON TOTE BAG,2,2.25,16686.0,15,female,21032.747844,United Kingdom
3,2011-05-29 14:08:00+00:00,554992,22617,BAKING SET SPACEBOY DESIGN,2,4.95,16686.0,15,female,21032.747844,United Kingdom


<p style='color:green; font-weight: 800;'>According to the received data, the Quantity field with a negative value does not refer to the entire order (InvoiceNo), but contains only some items. At the same time, the InvoiceDate of the initial order and the InvoiceDate of the InvoiceNo with a negative value of the quantity may contain a difference of several months. According to the conducted analysis, we make a decision to remove lines with negative values ​​of the Quantity field and not take these data into account in further analysis.</p>

#### Let's check UnitPrice column with negative values. 

In [14]:
select('8_unitprice_negative_values_check.sql')


,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
0,2011-08-12 14:51:00+00:00,A563186,B,Adjust bad debt,1,-11062.06,NaN,25,female,32130.475292,United Kingdom
1,2011-08-12 14:52:00+00:00,A563187,B,Adjust bad debt,1,-11062.06,NaN,25,female,32130.475292,United Kingdom


<p style='color:green; font-weight: 800;'>As we can see, there are only 2 entries that are bad debt adjustments. Thus, these lines are also subject to removal before further analysis.</p>

#### Let's check is there any cases when UnitPrice equal to 0. 

In [15]:
select('9_unitprice_equal_zero_check.sql')


,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
0,2010-12-01 11:52:00+00:00,536414,22139,None,56,0.0,NaN,25,female,32130.475292,United Kingdom
1,2010-12-01 14:32:00+00:00,536545,21134,None,1,0.0,NaN,25,female,32130.475292,United Kingdom
2,2010-12-01 14:33:00+00:00,536546,22145,None,1,0.0,NaN,25,female,32130.475292,United Kingdom
3,2010-12-01 14:33:00+00:00,536547,37509,None,1,0.0,NaN,25,female,32130.475292,United Kingdom
4,2010-12-01 14:34:00+00:00,536549,85226A,None,1,0.0,NaN,25,female,32130.475292,United Kingdom
...,...,...,...,...,...,...,...,...,...,...,...
2510,2011-11-17 19:52:00+00:00,577129,22464,HANGING METAL HEART LANTERN,4,0.0,15602.0,51,female,43011.013136,United Kingdom
2511,2011-04-04 14:42:00+00:00,548871,22162,HEART GARLAND RUSTIC PADDED,2,0.0,14410.0,54,male,42315.692977,United Kingdom
2512,2011-11-03 13:24:00+00:00,574252,M,Manual,1,0.0,12437.0,56,female,41268.288736,France
2513,2011-05-12 15:21:00+00:00,553000,47566,PARTY BUNTING,4,0.0,17667.0,57,female,39555.066691,United Kingdom


<p style='color:green; font-weight: 800;'>We have 2515 rows where UnitPrice is equal to 0. We can assume that this is a malfunction of the database, since the presence of records indicates that the sale took place.</p>

Additionaly, checking is there any cases, when UnitPrice equal to 0 and CustomerID is not NULL to see if this failure is related to a failure to write information about CustomerId.

In [16]:
select('10_unitprice_equal_zero_customerid_not_null.sql').head()


,InvoiceDate,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Age,Gender,Income,Country
0,2011-11-07 13:22:00+00:00,574879,22625,RED KITCHEN SCALES,2,0.0,13014.0,26,female,32215.022602,United Kingdom
1,2011-11-04 11:55:00+00:00,574469,22385,JUMBO BAG SPACEBOY DESIGN,12,0.0,12431.0,28,female,32569.958680,Australia
2,2010-12-05 14:02:00+00:00,537197,22841,ROUND CAKE TIN VINTAGE GREEN,1,0.0,12647.0,28,female,33274.191349,Germany
3,2010-12-16 14:36:00+00:00,539263,22580,ADVENT CALENDAR GINGHAM SACK,4,0.0,16560.0,28,female,32650.674167,United Kingdom
4,2011-09-27 09:46:00+00:00,568384,M,Manual,1,0.0,12748.0,29,male,31798.776856,United Kingdom


<p style='color:green; font-weight: 900;'>The results show that the errors are independent because we have 39 records where the Unit Price equal to 0 but the CustomerId is present in the record</p>

Let's check if there are cases where the UnitPrice equal to 0 but in the same period the same products were sold with the specified price to decide if we can replace the 0 price value with the one present in other orders.

In [17]:
select('11_check_unitprice_zero_replace_with_other_orders.sql').head(10)


,InvoiceDate,StockCode,UnitPrice,InvoiceDate_1,StockCode_1,UnitPrice_1
0,2011-10-26 13:27:00+00:00,21208,0.39,2011-10-26 14:36:00+00:00,21208,0.0
1,2011-03-30 12:45:00+00:00,22055,1.65,2011-03-30 12:45:00+00:00,22055,0.0
2,2011-03-30 11:12:00+00:00,22055,1.65,2011-03-30 12:45:00+00:00,22055,0.0
3,2011-03-30 10:54:00+00:00,22055,1.45,2011-03-30 12:45:00+00:00,22055,0.0
4,2011-11-03 19:17:00+00:00,22065,0.39,2011-11-03 11:47:00+00:00,22065,0.0
5,2011-11-03 19:50:00+00:00,22065,0.39,2011-11-03 11:47:00+00:00,22065,0.0
6,2011-01-06 15:11:00+00:00,22090,6.77,2011-01-06 16:41:00+00:00,22090,0.0
7,2011-04-04 14:42:00+00:00,22162,2.95,2011-04-04 14:42:00+00:00,22162,0.0
8,2011-01-13 11:48:00+00:00,22168,8.50,2011-01-13 15:10:00+00:00,22168,0.0
9,2010-12-21 13:06:00+00:00,22423,25.49,2010-12-21 13:45:00+00:00,22423,0.0


<p style='color:green; font-weight: 800;'>Indeed, there are cases when, on the same day, the same product was sold with the specified UnitPrice and with a UnitPrice equal to 0.</p>

Let's delve into this situation using the example of one specific product.

In [18]:
select('12_additional_checking_on_sample.sql').head(20)


,InvoiceDate,StockCode,UnitPrice
0,2010-12-01 12:27:00+00:00,22423,10.95
1,2010-12-01 12:36:00+00:00,22423,12.75
2,2010-12-01 12:54:00+00:00,22423,12.75
3,2010-12-01 13:17:00+00:00,22423,12.75
4,2010-12-01 13:21:00+00:00,22423,12.75
5,2010-12-01 13:51:00+00:00,22423,12.75
6,2010-12-01 14:03:00+00:00,22423,12.75
7,2010-12-01 14:32:00+00:00,22423,12.72
8,2010-12-01 15:08:00+00:00,22423,12.75
9,2010-12-01 17:06:00+00:00,22423,12.72


<p style='color:green; font-weight: 800;'>On the example of the product with the StockCode 22423, we can see that even on the same day, the product was sold with different prices. Based on this, we cannot decide to fill the 0 UnitPrice with another value. Therefore, lines with UnitPrice ​​equal to 0 will also be removed from the dataset before further investigation.</p>

#### Taking into account the previous analysis, we decide to create a new data set 'clean_customers'  
It will not contain rows in which:  
- CustomerId is NULL  
- Quantity < 0  
- Unit price <= 0  

In addition, the data type of the CustomerId column will be changed from float to string, and the date format of the InvoiceDate column will be changed to yyyy-mm-dd.  

Additionally, 2 new columns will be created:  
-Amount (Quantity * Price per unit)  
- Cohort (clients will be divided into cohorts according to the month in which they made their first purchase)  

In [19]:
select('13_create_new_table_clean_customers.sql')


sql_check_table = """
SELECT table_name
FROM `online-retail-rfm-cohort.online_retail.INFORMATION_SCHEMA.TABLES`
WHERE table_name = 'clean_customer'
"""

query_job_check = client.query(sql_check_table)

results = query_job_check.result()

if results.total_rows > 0:
    print("Table 'clean_customer' was successfully created.")
else:
    print("Table 'clean_customer' was not found.")
    

Table 'clean_customer' was successfully created.


Let's take a look on our cleaned data  

In [20]:
select('14_select_all_f_clean_customers.sql')


,Date,InvoiceNo,StockCode,Description,Quantity,UnitPrice,Amount,Customer_ID,Age,Gender,Income,Country,Cohort
0,2011-01-18,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1.04,77183.60,12346,37,female,33199.291996,United Kingdom,2
1,2010-12-07,537626,84969,BOX OF 6 ASSORTED COLOUR TEASPOONS,6,4.25,25.50,12347,73,female,31388.641266,Iceland,1
2,2010-12-07,537626,22728,ALARM CLOCK BAKELIKE PINK,4,3.75,15.00,12347,73,female,31388.641266,Iceland,1
3,2010-12-07,537626,20782,CAMOUFLAGE EAR MUFF HEADPHONES,6,5.49,32.94,12347,73,female,31388.641266,Iceland,1
4,2010-12-07,537626,84997B,RED 3 PIECE RETROSPOT CUTLERY SET,6,3.75,22.50,12347,73,female,31388.641266,Iceland,1
5,2010-12-07,537626,22725,ALARM CLOCK BAKELIKE CHOCOLATE,4,3.75,15.00,12347,73,female,31388.641266,Iceland,1
6,2010-12-07,537626,84997D,PINK 3 PIECE POLKADOT CUTLERY SET,6,3.75,22.50,12347,73,female,31388.641266,Iceland,1
7,2010-12-07,537626,71477,COLOUR GLASS. STAR T-LIGHT HOLDER,12,3.25,39.00,12347,73,female,31388.641266,Iceland,1
8,2010-12-07,537626,84558A,3D DOG PICTURE PLAYING CARDS,24,2.95,70.80,12347,73,female,31388.641266,Iceland,1
9,2010-12-07,537626,85116,BLACK CANDELABRA T-LIGHT HOLDER,12,2.10,25.20,12347,73,female,31388.641266,Iceland,1


Checking the correctness of the calculation of the number of customers in each cohort in accordance with the total number of unique customers  

Number of unique customers

In [21]:
select('15_num_unique_customers.sql')


,unique_customers
0,4338


Number of unique customers in each cohort

In [22]:
select('16_num_unique_customers_each_cohort.sql')


,Cohort,num_customers
0,1,885
1,10,299
2,11,358
3,12,323
4,13,41
5,2,417
6,3,380
7,4,452
8,5,300
9,6,284


<p style='color:green; font-weight: 800;'>The number of unique customers in the dataset is 4338, as well as the number of records of unique customers divided into cohorts</p>

### Сalculating monthly active customers (MAO) from each cohort 

In [23]:
select('17_monthly_active_customers_each_cohort.sql').fillna(0)


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,885,324,286,340,321,352,321,309,313,350,331,445,235
1,2,0,417,92,111,96,134,120,103,101,125,136,152,49
2,3,0,0,380,71,71,108,103,94,96,106,94,116,26
3,4,0,0,0,452,68,114,90,101,76,121,104,126,39
4,5,0,0,0,0,300,64,61,63,59,68,65,78,22
5,6,0,0,0,0,0,284,54,49,49,59,66,75,27
6,7,0,0,0,0,0,0,242,42,38,64,56,81,23
7,8,0,0,0,0,0,0,0,188,34,39,42,51,21
8,9,0,0,0,0,0,0,0,0,169,35,42,41,21
9,10,0,0,0,0,0,0,0,0,0,299,70,90,34


<p style='color:green; font-weight: 800;'>Therefore, the 1st cohort is the largest in terms of the number of engaged users.  
    
<p style='color:green; font-weight: 800;'>Starting from the 2nd month of the store's operation, number of engaged users is at least half as small. This may be related to a change in the marketing company, for example, in terms of the used engagement channels. Perhaps the customer acquisition cost (CAC) was too high and a decision was made to change the channels used, which in turn affected the number of customers attracted. 

<p style='color:green; font-weight: 800;'>The smallest cohort of engaged customers is the 9th cohort (we do not consider cohort # 13 as it only includes data for 9 days in January 2012).
As this cohort was recruited in August 2011, it can be assumed that the drop in engaged usersis due to the holiday period.  

<p style='color:green; font-weight: 800;'>In addition, the value of monthly active customers (MAO) remains more stable both in terms of the relative number of engaged customers and in terms of the absolute number in the 1st cohort. Which may indicate that the engagement channel that was used at the very beginning was much better focused on the store's target audience.  

<p style='color:green; font-weight: 800;'>A significant decrease in the number of monthly active customers (MAO) after acquisition may also indicate that the store may have used a significant number of discounts on products or service (shipping) terms that were subsequently canceled. This could be the factor that influenced the attraction of customers who wanted to take advantage of the discounts.  

<p style='color:green; font-weight: 800;'>During the year, we can observe 3 months in which the number of monthly active customers (MAO) increases compared to previous periods for all cohorts. These are May 2011, September 2011 and December 2011.  

<p style='color:green; font-weight: 800;'>In May and September 2011, such changes can be explained either by the introduction of seasonal discounts, while in December 2011, the activity is most likely related to the Christmas and New Year holidays.</p>

### Сalculating monthly Purchase Count from each cohort

In [24]:
select('18_monthly_purchase_count_each_cohort.sql').fillna(0)


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,1400,517,458,582,504,651,572,547,538,641,628,945,335
1,2,0,470,117,149,125,186,161,145,135,163,191,246,65
2,3,0,0,422,90,94,147,124,118,110,140,120,166,30
3,4,0,0,0,500,87,153,123,123,99,145,140,194,47
4,5,0,0,0,0,339,90,72,72,69,76,75,104,27
5,6,0,0,0,0,0,328,71,60,64,82,80,104,30
6,7,0,0,0,0,0,0,270,53,45,79,70,117,27
7,8,0,0,0,0,0,0,0,213,38,51,48,70,22
8,9,0,0,0,0,0,0,0,0,182,42,59,53,22
9,10,0,0,0,0,0,0,0,0,0,336,94,127,38


<p style='color:green; font-weight: 800;'>For a more convenient analysis, it makes sense to use a metric Usage Frequency</p>

### Сalculating monthly Usage Frequency from each cohort

In [25]:
select('19_monthly_usage_frequency_each_cohort.sql') \
    .fillna(0) \
    .map(lambda x: '{:.2f}'.format(x) if x != 0 and not float(x).is_integer() else '{:.0f}'.format(x))


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,1.58,1.60,1.60,1.71,1.57,1.85,1.78,1.77,1.72,1.83,1.90,2.12,1.43
1,2,0,1.13,1.27,1.34,1.30,1.39,1.34,1.41,1.34,1.30,1.40,1.62,1.33
2,3,0,0,1.11,1.27,1.32,1.36,1.20,1.26,1.15,1.32,1.28,1.43,1.15
3,4,0,0,0,1.11,1.28,1.34,1.37,1.22,1.30,1.20,1.35,1.54,1.21
4,5,0,0,0,0,1.13,1.41,1.18,1.14,1.17,1.12,1.15,1.33,1.23
5,6,0,0,0,0,0,1.15,1.31,1.22,1.31,1.39,1.21,1.39,1.11
6,7,0,0,0,0,0,0,1.12,1.26,1.18,1.23,1.25,1.44,1.17
7,8,0,0,0,0,0,0,0,1.13,1.12,1.31,1.14,1.37,1.05
8,9,0,0,0,0,0,0,0,0,1.08,1.20,1.40,1.29,1.05
9,10,0,0,0,0,0,0,0,0,0,1.12,1.34,1.41,1.12


<p style='color:green; font-weight: 800;'>The 1st cohort proved to be the most effective this time, having the highest frequency of purchases over the entire period under study. This can be explained by correctly hitting the target audience when attracting customers. Or the fact that customers of the 1st cohort have unique promotional privileges compared to others.

<p style='color:green; font-weight: 800;'>It is interesting that after the first experience of purchasing goods, active customers increase the frequency of purchases in subsequent periods in absolutely all cohorts. This may indicate that these customers are the true target audience of the store. Therefore, it is necessary to study the "portrait" of such customers in order to study the target audience in the following marketing companies.

<p style='color:green; font-weight: 800;'>However, in general, increasing the frequency of purchases is not enough to scale the store's income and its further development.</p>

### Сalculating monthly Average Order Value (AOV) from each cohort

In [26]:
select('20_monthly_average_order_value_each_cohort.sql')\
    .fillna(0)\
    .map(lambda x: '{:.2f}'.format(x) if x != 0 and not float(x).is_integer() else '{:.0f}'.format(x))


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,22.22,27.27,26.86,27.19,21.19,28.14,28.34,27.43,29.25,33.47,33.99,23.64,25.84
1,2,0,26.88,25.10,20.97,31.23,22.48,26.28,25.24,25.49,19.07,22.33,19.73,19.78
2,3,0,0,17.87,20.85,21.46,19.36,17.69,16.98,22.17,22.90,18.79,22.18,23.50
3,4,0,0,0,17.59,21.14,22.69,18.02,21.11,19,22.03,19.99,16.81,13.20
4,5,0,0,0,0,16.95,21.03,19.49,18.74,19.55,15,15.25,15.97,12.34
5,6,0,0,0,0,0,20.48,17.34,22.25,20.90,18.59,14.12,17.02,234.53
6,7,0,0,0,0,0,0,23.98,16.29,19.95,20.45,15.35,16.71,13.22
7,8,0,0,0,0,0,0,0,14.96,23.53,11.79,13.02,10.88,11.68
8,9,0,0,0,0,0,0,0,0,16.52,13.16,12.53,15.88,17
9,10,0,0,0,0,0,0,0,0,0,18.81,12.29,14.15,14.27


<p style='color:green; font-weight: 800;'>Cohorts 1 through 6 have larger average receipts than cohorts 7 through 12. Thus, although the frequency of purchases is not significantly different between cohorts, cohorts 1 through 6 generate more revenue. Let's test this by calculating the monthly income from each cohort and the cumulative income from each cohort to calculate Lifetime Value</p>

### Сalculating monthly Revenue from each cohort

In [27]:
select('21_monthly_revenue_each_cohort.sql')\
    .fillna(0)\
    .map(lambda x: '{:.2f}'.format(x) if x != 0 and not float(x).is_integer() else '{:.0f}'.format(x))


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,19665.92,8835.72,7682.09,9244.22,6802.56,9903.90,9096.35,8476.30,9155.47,11712.81,11251.85,10517.68,6072.75
1,2,0,11208.69,2309.19,2327.50,2998.47,3012.11,3153.62,2599.98,2574.84,2384.24,3037.18,2998.47,969.28
2,3,0,0,6789.74,1480.24,1523.56,2091.19,1822.23,1595.74,2127.92,2427.59,1766.22,2573.36,611.10
3,4,0,0,0,7950.31,1437.54,2587.09,1621.63,2131.68,1444.19,2666.12,2078.93,2118.37,514.67
4,5,0,0,0,0,5085.97,1345.86,1189.05,1180.41,1153.34,1019.94,991.38,1245.42,271.55
5,6,0,0,0,0,0,5817.24,936.42,1090.10,1023.93,1096.72,931.63,1276.36,6332.43
6,7,0,0,0,0,0,0,5804.18,684,758.08,1308.74,859.33,1353.64,304.05
7,8,0,0,0,0,0,0,0,2812.02,800.10,459.70,546.75,555.11,245.37
8,9,0,0,0,0,0,0,0,0,2792.15,460.52,526.05,651.19,357.03
9,10,0,0,0,0,0,0,0,0,0,5624.98,860.05,1273.35,485.13


### Calculation of Cumulative Revenue from each cohort

In [28]:
select('22_cumulative_revenue_each_cohort.sql')\
    .fillna(0)\
    .map(lambda x: '{:.2f}'.format(x) if x != 0 and not float(x).is_integer() else '{:.0f}'.format(x))


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,19664.70,28500.18,36182.14,45426.74,52228.73,62134.01,71231.15,79707.02,88862.27,100576.77,111827.46,122347.26,128419.66
1,2,0,11208.96,13518.16,15845.83,18843.91,21856.23,25009.83,27609.55,30184.04,32567.79,35604.67,38603.63,39572.85
2,3,0,0,6790.60,8270.95,9794.61,11885.49,13707.56,15303.68,17432,19859.40,21625.66,24198.54,24809.54
3,4,0,0,0,7950.68,9388.20,11974.86,13596.66,15728.77,17172.77,19838.40,21917.36,24035.42,24550.22
4,5,0,0,0,0,5085,6430.92,7619.81,8800.43,9953.88,10973.88,11965.13,13210.79,13482.27
5,6,0,0,0,0,0,5816.32,6752.68,7842.93,8867.03,9963.84,10895.76,12172.26,18504.57
6,7,0,0,0,0,0,0,5803.16,6487.34,7245.44,8554.24,9413.84,10767.35,11071.41
7,8,0,0,0,0,0,0,0,2812.48,3612.50,4072.31,4619.15,5174.03,5419.31
8,9,0,0,0,0,0,0,0,0,2791.88,3252.48,3778.74,4429.82,4786.82
9,10,0,0,0,0,0,0,0,0,0,5624.19,6484.49,7757.99,8243.17


<p style='color:green; font-weight: 800;'>The 1st cohort shows rapid growth from almost 20,000 in the first month to 50,000 in four months and 100,000 in 8 months. The second cohort shows significantly lower results, failing to reach the 50,000 mark even after 11 months. The results of cohorts 3 and 4 are at the same level, which may indicate a change in the unsuccessful marketing strategy implemented in January 2011. All other cohorts failed to overcome even the threshold of the initial value of the first cohort during their time of interaction with the product.</p>

### Calculation of Average Revenue Per User (ARPU) from each cohort

In [29]:
select('23_average_revenue_per_user_each_cohort.sql')\
    .fillna(0)\
    .map(lambda x: '{:.2f}'.format(x) if x != 0 and not float(x).is_integer() else '{:.0f}'.format(x))


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,22.22,32.20,40.88,51.33,59.02,70.21,80.49,90.06,100.41,113.65,126.36,138.25,145.11
1,2,0,26.88,32.42,38,45.19,52.41,59.98,66.21,72.38,78.10,85.38,92.57,94.90
2,3,0,0,17.87,21.77,25.78,31.28,36.07,40.27,45.87,52.26,56.91,63.68,65.29
3,4,0,0,0,17.59,20.77,26.49,30.08,34.80,37.99,43.89,48.49,53.18,54.31
4,5,0,0,0,0,16.95,21.44,25.40,29.33,33.18,36.58,39.88,44.04,44.94
5,6,0,0,0,0,0,20.48,23.78,27.62,31.22,35.08,38.37,42.86,65.16
6,7,0,0,0,0,0,0,23.98,26.81,29.94,35.35,38.90,44.49,45.75
7,8,0,0,0,0,0,0,0,14.96,19.22,21.66,24.57,27.52,28.83
8,9,0,0,0,0,0,0,0,0,16.52,19.25,22.36,26.21,28.32
9,10,0,0,0,0,0,0,0,0,0,18.81,21.69,25.95,27.57


<p style='color:green; font-weight: 800;'>The average revenue per user decreases in each subsequent cohort, which indicates the need to change strategies for attracting and retaining customers.  
<p style='color:green; font-weight: 800;'>Now we have all the data to calculate the most important metric for the long term - Lifetime Value</p>

### Calculation of Lifetime Value (LTV) from each cohort 

In [30]:
select('24_lifetime_value_each_cohort.sql')\
    .fillna(0)\
    .map(lambda x: '{:.2f}'.format(x) if x != 0 and not float(x).is_integer() else '{:.0f}'.format(x))


,Cohort,2010_12,2011_01,2011_02,2011_03,2011_04,2011_05,2011_06,2011_07,2011_08,2011_09,2011_10,2011_11,2011_12
0,1,35.15,51.39,65.47,87.86,92.66,129.84,143.42,159.43,172.59,208.13,239.74,293.58,206.85
1,2,0,30.30,41.23,51.01,58.84,72.75,80.47,93.21,96.75,101.84,119.91,149.82,125.89
2,3,0,0,19.85,27.59,34.13,42.57,43.43,50.56,52.56,69.02,72.65,91.13,75.33
3,4,0,0,0,19.46,26.57,35.56,41.11,42.38,49.49,52.60,65.27,81.87,65.46
4,5,0,0,0,0,19.15,30.14,29.98,33.53,38.80,40.88,46.02,58.71,55.15
5,6,0,0,0,0,0,23.65,31.26,33.82,40.78,48.76,46.50,59.43,72.40
6,7,0,0,0,0,0,0,26.75,33.83,35.46,43.63,48.63,64.27,53.71
7,8,0,0,0,0,0,0,0,16.95,21.48,28.33,28.08,37.77,30.20
8,9,0,0,0,0,0,0,0,0,17.79,23.09,31.41,33.88,29.67
9,10,0,0,0,0,0,0,0,0,0,21.14,29.12,36.61,30.81


<p style='color:green; font-weight: 800;'>Analyzing the obtained figures, it can be concluded that, in the long run, the customers of the first and second cohorts will have greater value. Each subsequent cohort has an indicator that is at least twice as low as the indicator of the first two cohorts.
Thus, it is necessary to make a detailed analysis and segmentation of the customers of the first two cohorts and, perhaps, once again review the marketing strategy that was applied to attract the first cohort of customers, since it was the first cohort during the year that turned out to be the most numerous and the most effective by all metrics.</p>